# Ejercicios sobre pipelines

In [41]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score

# Cargar el conjunto de datos Iris
iris = load_iris(as_frame=True).frame
X = iris.drop(columns='target', axis=1)
y = iris['target']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Ejercicio 1

Crea un pipeline que tenga como tarea normalizar, para ello, utiliza la clase MinMaxScaler. Comprueba el resultado.

In [42]:
# Convertir datos categóricos a numéricos si es necesario
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

# Asegurarte de que ambas particiones tengan las mismas columnas
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# NO HACE FALTA PERO POR SI ACASO DA FALLO


In [43]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

# Crear el pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),       # Normalización
    ('pca', PCA(n_components=2)),    # Reducción de dimensionalidad
    ('svc', SVC(kernel='linear'))    # Clasificación con SVM
])

# Entrenar el pipeline con el conjunto de entrenamiento
pipeline.fit(X_train, y_train)

# Predecir los valores para el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Calcular y mostrar la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.2f}")

Precisión del modelo: 1.00


Crea un pipeline que tenga como tarea estandarizar, para ello, utiliza la clase StandardScaler. Comprueba el resultado. 

In [44]:
from sklearn.preprocessing import StandardScaler

# Crear el pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),    # Estandarización
    ('pca', PCA(n_components=2)),    # Reducción de dimensionalidad
    ('svc', SVC(kernel='linear'))    # Clasificación con SVM
])


In [ ]:
# Entrenar el pipeline con el conjunto de entrenamiento
pipeline.fit(X_train, y_train)

# Predecir los valores para el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Calcular y mostrar la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo con estandarización: {accuracy:.2f}")

# jaime
# pipeline2.fit_transform(X_train)

Precisión del modelo con estandarización: 0.90


## Ejercicio 2

Crea tu propio transformador que normalize un conjunto. Recuerda que la fórmula para normalizar es la siguiente:

```
xi_normalizada = xi - min(x) / max(ex) - min(ex)
```

Donde:
* *xi* es el valor a normalizar.
* *min(x)* es el mínimo del conjunto de los datos.
* *min(x)* es el máximo del conjunto de los datos.
* *xi_normalizada* es el valor de xi normalizado.


In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
import pandas as pd

class CustomNormalizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # Calcular los valores mínimos y máximos para cada característica
        self.min_ = X.min(axis=0)
        self.max_ = X.max(axis=0)
        return self

    def transform(self, X):
        return (X - self.min_) / (self.max_ - self.min_)

    def inverse_transform(self, X):
        # Revertir la normalización
        return X * (self.max_ - self.min_) + self.min_


Comprueba que el resultado es igual que el que el de la clase MinMaxScaler

In [2]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# jaime
pipeline3 = Pipeline([
    ('normalizacion', CustomNormalizer())
])

pipeline3.fit_transform(X_train)


NameError: name 'pd' is not defined

## Ejercicio 3

Crea tu propio transformador que estandarice un conjunto. Recuerda que la fórmula para estandarizar es la siguiente:

```
xi_estandarizada = (xi - media(x)) / std(x)
```

Donde:
* *xi* es el valor a estandarizar.
* *media(x)* es la media de los datos.
* *std(x)* es la desviación típica de los datos.
* *xi_estandarizada* es el valor de xi estandarizado.




In [48]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# Definir el transformador personalizado para estandarizar
class MiEstandarizador(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.media = X.mean()  # Calcula la media de cada columna
        self.std = X.std()     # Calcula la desviación estándar de cada columna
        return self

    def transform(self, X, y=None):
        # Establecer la fórmula para estandarizar cada valor
        X_estandarizado = (X - self.media) / self.std
        return X_estandarizado


In [1]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Crear un pipeline que incluya el estandarizador y un clasificador SVM
pipeline = Pipeline([
    ('estandarizador', MiEstandarizador()),  # Usamos nuestro transformador personalizado
    ('svm', SVC())                          # Modelo de clasificación
])

# Ajustar el pipeline con los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Predecir con el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# jaime
# pipeline4 = Pipeline([
#     ('estandarizacion2', TransformadorCrisis2())
# ])

# pipeline4.fit_transform(X_train)

NameError: name 'MiEstandarizador' is not defined

Comprueba que el resultado es igual que el de StandardScaler

In [50]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Aplicamos StandardScaler a X_train
scaler = StandardScaler()
X_train_standardized = scaler.fit_transform(X_train)

# Usamos nuestro transformador personalizado MiEstandarizador
mi_estandarizador = MiEstandarizador()
X_train_mi_estandarizado = mi_estandarizador.fit_transform(X_train)

# Comprobamos si los resultados son iguales (deberían serlo si todo funciona correctamente)
# Podemos usar np.allclose para comparar los resultados (debido a pequeñas diferencias numéricas)
resultados_iguales = np.allclose(X_train_standardized, X_train_mi_estandarizado)

# Imprimir el resultado
print(f"¿Los resultados son iguales? {resultados_iguales}")


¿Los resultados son iguales? False


## Ejercicio 4. 

Crea tu propio transformador, aplicará el mismo LabelEncoder a todas las columnas categóricas del siguiente dataset. 

In [51]:
# Cargar el conjunto de datos Iris
iris = load_iris(as_frame=True).frame
X = iris.drop(columns='target', axis=1)
y = iris['target']

# Calculo la mitad del dataset
midpoint = len(X) // 2

# Asigno valores diferentes a la mitad de la columna
X.loc[:midpoint, 'categorica1'] = 'valor1'
X.loc[midpoint:, 'categorica1'] = 'valor2'

# Asigno valores diferentes a la mitad de la columna
X.loc[:midpoint, 'categorica2'] = 'valor2'
X.loc[midpoint:, 'categorica2'] = 'valor1'


# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [52]:
from sklearn.base import BaseEstimator, TransformerMixin
# Imprime los datos de entrenamiento para ver el conjunto X_train
print(X_train)

# Definir un transformador personalizado heredando de BaseEstimator y TransformerMixin
class MiLabelEncoder(BaseEstimator, TransformerMixin):
    
    # Método fit para ajustar el transformador a los datos
    def fit(self, X, y=None):
        
        # Seleccionar solo las columnas categóricas (de tipo object)
        self.columnas_categoricas = list(X.dtypes[X.dtypes == object].index)

        # Inicializar una lista para almacenar los valores únicos de todas las columnas categóricas
        valores_unicos = list()
        for c in self.columnas_categoricas:
            # Obtener los valores únicos de cada columna y agregarlo a la lista
            valores_unicos.extend(X[c].unique())

        # Eliminar valores duplicados
        valores_unicos = np.unique(valores_unicos)

        # Crear un LabelEncoder y ajustarlo con los valores únicos de las columnas categóricas
        self.le = LabelEncoder()
        self.le.fit(valores_unicos)  # Ajustar el encoder a todos los valores posibles

        return self

    # Método transform para aplicar el ajuste a los datos
    def transform(self, X, y=None):
        # Iterar sobre cada columna categórica y transformarla
        for c in self.columnas_categoricas:
            # Usar el LabelEncoder ajustado para transformar los valores
            X[c] = self.le.transform(X[c])
        return X    


     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
22                 4.6               3.6                1.0               0.2   
15                 5.7               4.4                1.5               0.4   
65                 6.7               3.1                4.4               1.4   
11                 4.8               3.4                1.6               0.2   
42                 4.4               3.2                1.3               0.2   
..                 ...               ...                ...               ...   
71                 6.1               2.8                4.0               1.3   
106                4.9               2.5                4.5               1.7   
14                 5.8               4.0                1.2               0.2   
92                 5.8               2.6                4.0               1.2   
102                7.1               3.0                5.9               2.1   

    categorica1 categorica2

In [53]:
# Instanciar el transformador personalizado
mlbe = MiLabelEncoder()

# Ajustar y transformar el conjunto de entrenamiento X_train
mlbe.fit_transform(X_train)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),categorica1,categorica2
22,4.6,3.6,1.0,0.2,0,1
15,5.7,4.4,1.5,0.4,0,1
65,6.7,3.1,4.4,1.4,0,1
11,4.8,3.4,1.6,0.2,0,1
42,4.4,3.2,1.3,0.2,0,1
...,...,...,...,...,...,...
71,6.1,2.8,4.0,1.3,0,1
106,4.9,2.5,4.5,1.7,1,0
14,5.8,4.0,1.2,0.2,0,1
92,5.8,2.6,4.0,1.2,1,0


In [54]:
# OTRO EJEMPLO SIN LABELENCODER

from sklearn.base import BaseEstimator, TransformerMixin
# Imprime los datos de entrenamiento para ver el conjunto X_train
print(X_train)

# Definir un transformador personalizado heredando de BaseEstimator y TransformerMixin
class MiLabelEncoder2(BaseEstimator, TransformerMixin):
    
    # Método fit para ajustar el transformador a los datos
    def fit(self, X, y=None):
        
        # Seleccionar solo las columnas categóricas (de tipo object)
        self.columnas_categoricas = list(X.dtypes[X.dtypes == object].index)

        # Inicializar una lista para almacenar los valores únicos de todas las columnas categóricas
        valores_unicos = list()
        for c in self.columnas_categoricas:
            # Obtener los valores únicos de cada columna y agregarlo a la lista
            valores_unicos.extend(X[c].unique())

        # Eliminar valores duplicados
        valores_unicos = np.unique(valores_unicos)
        
        # Mapeo sin LabelEncoder
        self.mapeo={}
        i=0
        for v in valores_unicos:
            self.mapeo[v]=i
            i=i+1
        return self

    # Método transform para aplicar el ajuste a los datos
    def transform(self, X, y=None):
        # Iterar sobre cada columna categórica y transformarla
        for c in self.columnas_categoricas:
            # Usamos el map
            X[c] = X[c].map(self.mapeo)
        return X    


     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
22                 4.6               3.6                1.0               0.2   
15                 5.7               4.4                1.5               0.4   
65                 6.7               3.1                4.4               1.4   
11                 4.8               3.4                1.6               0.2   
42                 4.4               3.2                1.3               0.2   
..                 ...               ...                ...               ...   
71                 6.1               2.8                4.0               1.3   
106                4.9               2.5                4.5               1.7   
14                 5.8               4.0                1.2               0.2   
92                 5.8               2.6                4.0               1.2   
102                7.1               3.0                5.9               2.1   

     categorica1  categoric

In [55]:
# UTILIZO UN LABEL ENCODER DIFERENTE PARA CADA COLUMNA CATEGORICA

from sklearn.base import BaseEstimator, TransformerMixin
# Imprime los datos de entrenamiento para ver el conjunto X_train
print(X_train)

# Definir un transformador personalizado heredando de BaseEstimator y TransformerMixin
class MiLabelEncoder3(BaseEstimator, TransformerMixin):
    
    # Método fit para ajustar el transformador a los datos
    def fit(self, X, y=None):
        
        # Seleccionar solo las columnas categóricas (de tipo object)
        columnas_categoricas = list(X.dtypes[X.dtypes == object].index)

        self.label_encoders = {}
        for c in columnas_categoricas:
            self.label_encoders[c] = LabelEncoder()
            self.label_encoders[c].fit(X[c])

        return self

    # Método transform para aplicar el ajuste a los datos
    def transform(self, X, y=None):
        # Iterar sobre cada columna categórica y transformarla
        for c in self.columnas_categoricas:
            # Usar el LabelEncoder ajustado para transformar los valores
            X[c] = self.le.transform(X[c])
        return X    

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
22                 4.6               3.6                1.0               0.2   
15                 5.7               4.4                1.5               0.4   
65                 6.7               3.1                4.4               1.4   
11                 4.8               3.4                1.6               0.2   
42                 4.4               3.2                1.3               0.2   
..                 ...               ...                ...               ...   
71                 6.1               2.8                4.0               1.3   
106                4.9               2.5                4.5               1.7   
14                 5.8               4.0                1.2               0.2   
92                 5.8               2.6                4.0               1.2   
102                7.1               3.0                5.9               2.1   

     categorica1  categoric

## Ejercicio 5

Crea tu propio transformador para aplicar la técnica de one_hot_encoding a un conjunto de columnas que pasaré como parámetro. 

In [56]:

from sklearn.base import BaseEstimator, TransformerMixin
# Imprime los datos de entrenamiento para ver el conjunto X_train
print(X_train)

# Definir un transformador personalizado heredando de BaseEstimator y TransformerMixin
class MiOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self,columns):
        self.columns = columns
    # Método fit para ajustar el transformador a los datos
    def fit(self, X, y=None):
        
        return self

    # Método transform para aplicar el ajuste a los datos
    def transform(self, X):
        for col in self.columns:
            X = pd.concat([X,pd.get_dummies(X[col], prefix=col)], axis=1)
            X = X.drop(col,axis=1)
        return X    


     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
22                 4.6               3.6                1.0               0.2   
15                 5.7               4.4                1.5               0.4   
65                 6.7               3.1                4.4               1.4   
11                 4.8               3.4                1.6               0.2   
42                 4.4               3.2                1.3               0.2   
..                 ...               ...                ...               ...   
71                 6.1               2.8                4.0               1.3   
106                4.9               2.5                4.5               1.7   
14                 5.8               4.0                1.2               0.2   
92                 5.8               2.6                4.0               1.2   
102                7.1               3.0                5.9               2.1   

     categorica1  categoric

In [57]:
# Instanciar el transformador personalizado
mlbe = MiOneHotEncoder(columns=['categorica1'])

# Ajustar y transformar el conjunto de entrenamiento X_train
mlbe.fit_transform(X_train)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),categorica2,categorica1_0,categorica1_1
22,4.6,3.6,1.0,0.2,1,True,False
15,5.7,4.4,1.5,0.4,1,True,False
65,6.7,3.1,4.4,1.4,1,True,False
11,4.8,3.4,1.6,0.2,1,True,False
42,4.4,3.2,1.3,0.2,1,True,False
...,...,...,...,...,...,...,...
71,6.1,2.8,4.0,1.3,1,True,False
106,4.9,2.5,4.5,1.7,0,False,True
14,5.8,4.0,1.2,0.2,1,True,False
92,5.8,2.6,4.0,1.2,0,False,True


Aplica el transformador a las columnas categóricas del siguiente dataset

In [58]:
# Cargar el conjunto de datos Iris
iris = load_iris(as_frame=True).frame
X = iris.drop(columns='target', axis=1)
y = iris['target']

# Calculo la mitad del dataset
midpoint = len(X) // 2

# Asigno valores diferentes a la mitad de la columna
X.loc[:midpoint, 'categorica1'] = 'valor1'
X.loc[midpoint:, 'categorica1'] = 'valor2'

# Asigno valores diferentes a la mitad de la columna
X.loc[:midpoint, 'categorica2'] = 'valor2'
X.loc[midpoint:, 'categorica2'] = 'valor1'


# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ejercicio 6

Crea un transformador llamado MultiplierTransformer que tome un conjunto de datos y multiplique todas las características numéricas por un factor específico. 

In [ ]:
class Transformador5(BaseEstimator, TransformerMixin):

    def __init__(self, valor):
      self.valor = valor

    def fit(self, X, y=None):
      return self

    def transform(self, X):
      X = X * self.valor
      return X

Escribe un ejemplo de uso utilizando el siguiente dataset:

In [59]:
# Ejemplo de uso con un DataFrame de pandas
import pandas as pd

data = {'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]}
df = pd.DataFrame(data)

In [ ]:
pipeline7 = Pipeline([
    ('TransformadorAntiVacio', Transformador5())
])

pipeline7.fit_transform(df)

# Ejercicio 7

Desarrolla un transformador llamado MissingDataImputerNumerical que sustituya los datos faltantes en las columnas por el valor de la media de cada columna. 

In [60]:
columnas_numericas=list(df.dtypes[df.dtypes[df.dtypes!='object'].index])

df[columnas_numericas].mean()


KeyError: "None of [Index([int64, int64, int64], dtype='object')] are in the [columns]"

In [29]:
from sklearn.base import BaseEstimator, TransformerMixin

class MissingDataImputerNumerical(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        columnas_numericas=list(X.dtypes[X.dtypes[X.dtypes!='object'].index])
        self.medias= X[columnas_numericas].mean()
        return self
    
    def transform(self, X):
        return X.fillna(self.medias)
    
    # otra forma
    # X.replace(np.nan, self.medias)

    # Otra forma es en el fit solo poner 
    # self.medias = X.meal()
    # return self

Ejecuta un ejemplo utilizando el siguiente dataset

In [ ]:
import pandas as pd
import numpy as np

data = {'A': [1, 2, np.nan], 'B': [4, np.nan, 6], 'C': [7, 8, 9]}
df = pd.DataFrame(data)

In [ ]:
mdin = MissingDataImputerNumerical()
mdin.fit_transform(df)

KeyError: "None of [Index([float64, float64, int64], dtype='object')] are in the [columns]"

# Ejercicio 8

Desarrolla un transformador llamado MissingDataImputerCategorical que sustituya los datos faltantes en las columnas categóricas por el valor que más veces aparezca. 

In [ ]:
# Sacamos el valor que mas se repite
    
df['D'].value_counts().sort_values(ascending=False).index[0]
# Otra forma
# df['D'].value_counts().idxmax()
# Otra forma
# df['D'].mode()


from sklearn.base import BaseEstimator, TransformerMixin

class MissingDataImputerCategorical(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.modas= {}
        for c in X.columns:
            self.modas[c]= X[c].mode().iloc[0]
        return self
    
    def transform(self, X):
        return X.fillna(self.modas)



Ejecuta un ejemplo utilizando el siguiente dataset

In [31]:
import pandas as pd

data = {'A': [1, 2, np.nan, 4], 'B': [4, np.nan, 6, 5], 'C': [7, 8, 9, 10], 'D': ['a', 'a', np.nan,'b'], 'E': ['x', 'c', np.nan,'x']}
df = pd.DataFrame(data)
df

,A,B,C,D,E
0,1.0,4.0,7,a,x
1,2.0,NaN,8,a,c
2,NaN,6.0,9,NaN,NaN
3,4.0,5.0,10,b,x


In [3]:
mdic= MissingDataImputerCategorical()
mdic.fit_transform(df)

NameError: name 'MissingDataImputerCategorical' is not defined

In [ ]:
# CUANDO USAR FIT Y CUANDO TRANSFORM

#fit:

    #Se usa solo con X_train.
    #Su propósito es aprender los parámetros del transformador (como la media y desviación estándar para la normalización o los rangos para escalado).
    #No debe usarse con X_test, ya que X_test debe mantenerse independiente y solo transformarse usando los parámetros aprendidos de X_train.

#transform:

    #Se usa con X_train y X_test después de usar fit con X_train.
    #Aplica la transformación basada en los parámetros aprendidos durante el fit en X_train.
    #Específicamente:
    #X_train: Transformar después de fit para que los datos de entrenamiento sean procesados correctamente.
    #X_test: Transformar usando los parámetros aprendidos en X_train.
#
#fit_transform:

    #Es un atajo para hacer fit seguido de transform.
    #Se usa solo con X_train para ajustar y transformar los datos de una vez.


SyntaxError: invalid syntax (2025738096.py, line 1)